In [7]:
# setting up selenium/ beautifulsoup/ pandas
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

1. Create two functions, one for scrapping normal pages and the other for Alaska and American Samoa 
2. Loop over all pages to scrape required data
3. After collecting all data, append to one list, then create dataframe object

In [8]:
# Set up selenium and soup 
service = Service() 
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# options = webdriver.ChromeOptions()
# options.add_experimental_option("detach", True)
# service = Service(executable_path="chromedriver.exe")
# driver = webdriver.Chrome(service=service, options=options)

The chromedriver version (123.0.6312.58) detected in PATH at c:\Users\quent\repos\ISOM3400\Assignments\chromedriver.exe might not be compatible with the detected chrome version (124.0.6367.63); currently, chromedriver 124.0.6367.91 is recommended for chrome 124.*, so it is advised to delete the driver in PATH and retry


In [9]:
# Function for scrapping normal pages 
def scrape_election_page():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # WebDriverWait(driver, 5).until(
    #         EC.presence_of_element_located((By.CSS_SELECTOR, 'div.tile-h_ymFU.cnn-pcl-13b0kh1'))
    #     )
    columns = soup.select('div.tile-h_ymFU.cnn-pcl-13b0kh1[data-testid="card"]')
    county_name = ''

# Add in delegate numbers from table above
    delegate_rows = soup.select('tr.cnn-pcl-1me6450.isWinner-3g_AYM')
    candidate_delegates = {} 
    for row in delegate_rows:
        name_element = row.select_one('span[data-testid="candidate-name"]')
        candidate = name_element.get_text(strip=True) if name_element else None

        delegate_element = row.select_one('td[data-testid="delegates"]')
        delegates = int(delegate_element.get_text(strip=True)) if delegate_element else 0

        # Initialize the candidate in the dictionary if it doesn't exist
        if candidate not in candidate_delegates:
            candidate_delegates[candidate] = 0
            
        # Store the candidate-delegate pair in the dictionary
        candidate_delegates[candidate] += delegates


    for column in columns: 
        county_name = soup.select_one('article.core-result div.header-container-1LzJY9 h2').text
        table = column.select_one('table.cnn-pcl-1me6450')
        rows = table.select('tr.cnn-pcl-1me6450')

        for row in rows:
            state = soup.select_one('h2.header-2-AOgLYo.cnn-pcl-xk8c6r').get_text().split()[-1]
            county = county_name
            name_element = row.select_one('span[data-testid="candidate-name"]')
            candidate = name_element.get_text(strip=True) if name_element else None

            party_element = row.select_one('span[data-testid="party-label"]')
            party = party_element.get_text(strip=True).split(',')[0] if party_element else None

            incumbent = 'Incumbent' in party_element.get_text(strip=True) if party_element else False
            incumbent_status = 'Yes' if incumbent else 'No'

            vote_percent_element = row.select_one('td[data-testid="votepercent"]')
            percentage = vote_percent_element.get_text(strip=True) if vote_percent_element else None
            
            vote_count_element = row.select_one('span[data-testid="votes"]')
            votes = None
            if vote_count_element and not '':
                votes = int(vote_count_element.get_text(strip=True).replace(',', '')) if vote_count_element else None
            

            winner = 'Yes' if incumbent_status == 'Yes' else 'No'

            delegates = candidate_delegates.get(candidate, 0)

            if candidate and votes:
                yield {
                    'State': state,
                    'County': county_name,
                    'Candidate': candidate,
                    'Party': party,
                    'Incumbent': incumbent_status,
                    'Votes': votes,
                    'Percentage': percentage,
                    'Winner': winner,
                    'Delegates': delegates
                }


In [10]:
# Function for scrapping the two special pages 
def scrape_election_page_special():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    columns = soup.select('article.core-result')
    
    for column in columns:
        print(column.prettify())
        print('-' * 80)
        county_name = None
        table = column.select_one('table.cnn-pcl-1me6450')
        rows = table.select('tr.cnn-pcl-1me6450')

        for row in rows:
            candidate_delegates = {} 
            name_element = row.select_one('span[data-testid="candidate-name"]')
            candidate = name_element.get_text(strip=True) if name_element else None

            delegate_element = row.select_one('td[data-testid="delegates"]')
            try:
                delegates = int(delegate_element.get_text(strip=True)) if delegate_element else 0
            except ValueError:
            # Handle the case where the text cannot be converted to an integer
                    delegates = 0

            # Initialize the candidate in the dictionary if it doesn't exist
            if candidate not in candidate_delegates:
                candidate_delegates[candidate] = 0
                
            # Store the candidate-delegate pair in the dictionary
            candidate_delegates[candidate] += delegates


            state = soup.select_one('h2.header-2-AOgLYo.cnn-pcl-xk8c6r').get_text().split()[-1]
            party_element = row.select_one('span[data-testid="party-label"]')
            party = party_element.get_text(strip=True).split(',')[0] if party_element else None

            incumbent = 'Incumbent' in party_element.get_text(strip=True) if party_element else False
            incumbent_status = 'Yes' if incumbent else 'No'

            vote_percent_element = row.select_one('td[data-testid="votepercent"]')
            percentage = vote_percent_element.get_text(strip=True) if vote_percent_element else None
            
            vote_count_element = row.select_one('span[data-testid="votes"]')
            votes = None
            if vote_count_element and not '':
                votes = int(vote_count_element.get_text(strip=True).replace(',', '')) if vote_count_element else None
        
            winner = 'Yes' if incumbent_status == 'Yes' else 'No'

            delegates = candidate_delegates.get(candidate, 0)

            if party:
                yield {
                    'State': state,
                    'County': county_name,
                    'Candidate': candidate,
                    'Party': party,
                    'Incumbent': incumbent_status,
                    'Votes': votes,
                    'Percentage': percentage,
                    'Winner': winner,
                    'Delegates': delegates
                }

In [11]:
states_democratic = ['Alabama', 'American Samoa', 'Arkansas', 'California', 'Colorado', 
                     'Iowa', 'Maine', 'Massachusetts', 'Minnesota', 'North Carolina',
                     'Oklahoma', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia']
states_republican = ['Alabama', 'Alaska', 'Arkansas', 'California', 'Colorado',
                     'Maine', 'Massachusetts', 'Minnesota', 'North Carolina', 
                     'Oklahoma', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia']
states_both = [state for state in states_democratic if state in states_republican]
states_both.insert(4, 'Iowa')
print(states_both)


['Alabama', 'Arkansas', 'California', 'Colorado', 'Iowa', 'Maine', 'Massachusetts', 'Minnesota', 'North Carolina', 'Oklahoma', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia']


In [12]:
prefix_dem = "https://edition.cnn.com/election/2024/primaries-and-caucuses/results/"
suffix_dem = "/democratic-presidential-primary"
prefix_rep = "https://edition.cnn.com/election/2024/primaries-and-caucuses/results/"
suffix_rep = "/republican-presidential-primary"

combined_data = [] 
columns = ['State', 'County', 'Candidate', 'Party', 'Incumbent', 'Votes', 'Percentage', 'Winner', 'Delegates']
combined_df = pd.DataFrame(columns=columns)
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Append data for Democratic Party 
for i in states_both:
    # modify the state names
    state_name = i.lower().replace(" ", "-")
    link = f'{prefix_dem}{state_name}{suffix_dem}'
    driver.get(link)

    wait = WebDriverWait(driver, 1)  
    button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.rightButton.cnn-pcl-13b0kh1")))

    # Right-click button element 
    have_button = True
    button = driver.find_element(By.CSS_SELECTOR, "button.rightButton.cnn-pcl-13b0kh1")

    # Display all pages 
    while have_button: 
        try:
            button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.rightButton.cnn-pcl-13b0kh1")))
            # Scrape the election page and add data to combined list
            for election_data in scrape_election_page():
                    if election_data is not None:
                        combined_data.append(election_data) 
            button.click()
        except TimeoutException:
            have_button = False


# Append data for Republican Party 
for i in states_both:
    # modify the state names
    state_name = i.lower().replace(" ", "-")
    link = f'{prefix_rep}{state_name}{suffix_rep}'
    driver.get(link)

    wait = WebDriverWait(driver, 1)  
    button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.rightButton.cnn-pcl-13b0kh1")))

    # Right-click button element 
    have_button = True
    button = driver.find_element(By.CSS_SELECTOR, "button.rightButton.cnn-pcl-13b0kh1")

    # Display all pages 
    while have_button: 
        try:
            button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.rightButton.cnn-pcl-13b0kh1")))
            # Scrape the election page and add data to combined list
            for election_data in scrape_election_page():
                    if election_data is not None:
                        combined_data.append(election_data)
            button.click()
        except TimeoutException:
            have_button = False

    
# Append data for the two special webpages (Alaska and American Samoa)
states_special = ['Alaska', 'American Samoa'] 
for i in states_special: 
        state_name = i.lower().replace(" ", "-")
        prefix = "https://edition.cnn.com/election/2024/primaries-and-caucuses/results/"
        link = f'{prefix}{state_name}'
        driver.get(link)
        for election_data in scrape_election_page_special():
                        if election_data is not None:
                            combined_data.append(election_data)


<article class="core-result cnn-pcl-1hnl3l7 called-r39uAo" data-mode="during" style="--winnerPartyColor: #186CFD;">
 <div class="header-container-1LzJY9 cnn-pcl-xk8c6r has-follow-1yIbzu">
  <h2 class="header-2-AOgLYo cnn-pcl-xk8c6r">
   Democratic Presidential Caucuses: Alaska
  </h2>
  <div class="action-button-2bIakM cnn-pcl-xk8c6r">
  </div>
  <div aria-label="editorial-content" class="core-result__editorial-content-2hv5GK cnn-pcl-1wwonlt">
   Alaska Democrats are holding a voice vote at local caucuses, and President Joe Biden is the only candidate on the ballot.
  </div>
  <span class="core-result__total-counts-y78OQg cnn-pcl-1ksfwcp">
   <div class="circular_summary-LUtCUF wrapper cnn-pcl-eyhmk3" data-testid="pcl-circular_summary">
    <div class="circular_summary__primary_text-36YSkr cnn-pcl-eyhmk3" data-testid="pcl-circular_summary__primary_text">
     15
    </div>
    <div class="circular_summary__secondary_text-1rcr5S cnn-pcl-eyhmk3" data-testid="pcl-circular_summary__seconda

In [13]:
combined_df = pd.DataFrame(combined_data)     
print(combined_df.shape)  
combined_df.reset_index(inplace=True, drop=True) # <-- Reset the index of the combined dataFrame
combined_df.index = combined_df.index + 1 
combined_df = combined_df.fillna(value=np.nan)
combined_df['Votes'] = combined_df['Votes'].fillna(0).astype('int')

# combined_df.to_excel('data.xlsx', sheet_name='Assignment_1_Sheet', index=False)  <-- For self-reference
combined_df.to_csv('data2.csv', index=False)

(10964, 9)
